In [1]:
!pip install -q bertopic[visualization] sentence-transformers umap-learn hdbscan transformers torch datasets nltk matplotlib wordcloud pycountry pandas

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 46.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 35.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 30.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 47.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 39.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 153.0/153.0 kB 12.8 MB/s eta 0:00:00


In [2]:
!pip install streamlit transformers torch

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 50.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 71.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 8.8 MB/s eta 0:00:00


In [7]:
%%writefile app.py
import streamlit as st
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import nltk
import torch
import requests
from io import StringIO
import pycountry
from bertopic import BERTopic
from umap import UMAP
from hdbscan import HDBSCAN
from wordcloud import WordCloud
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer
from sentence_transformers import SentenceTransformer
from transformers import AutoTokenizer, AutoModelForSequenceClassification, pipeline

nltk.download('stopwords')

# === Setup === #
@st.cache_resource
def load_resources():
    # Stopwords and City/Country
    url = "https://raw.githubusercontent.com/datasets/world-cities/master/data/world-cities.csv"
    df_cities = pd.read_csv(StringIO(requests.get(url).text))
    city_names = df_cities['name'].str.lower().unique().tolist()
    country_names = [country.name.lower() for country in pycountry.countries]
    custom_stopwords = list(set(
        stopwords.words('english') + city_names + country_names +
        ['flight', 'airline', 'seat', 'boarding', 'ticket', 'airport', 'staff', 'delta', 'emirates', 'lufthansa', 'united']
    ))

    # Models
    embedding_model = SentenceTransformer("all-MiniLM-L6-v2")
    vectorizer_model = CountVectorizer(stop_words=custom_stopwords, min_df=5)
    umap_model = UMAP(n_neighbors=15, n_components=5, min_dist=0.0, metric='cosine', random_state=42)
    hdbscan_model = HDBSCAN(min_cluster_size=10, metric='euclidean', prediction_data=True)

    topic_model = BERTopic(
        embedding_model=embedding_model,
        vectorizer_model=vectorizer_model,
        umap_model=umap_model,
        hdbscan_model=hdbscan_model,
        language="english",
        calculate_probabilities=True,
        verbose=False
    )

    sentiment_pipe = pipeline(
        "sentiment-analysis",
        model=AutoModelForSequenceClassification.from_pretrained("cardiffnlp/twitter-roberta-base-sentiment"),
        tokenizer=AutoTokenizer.from_pretrained("cardiffnlp/twitter-roberta-base-sentiment"),
        device=0 if torch.cuda.is_available() else -1
    )

    return topic_model, sentiment_pipe

# === Load data === #
@st.cache_data
def load_data():
    df = pd.read_csv("airline_reviews_cleaned_dataset.csv")
    df = df[['customer_review']].dropna()
    df['customer_review'] = df['customer_review'].astype(str).str[:800]
    return df

# === Sentiment Mapper === #
label_map = {"LABEL_0": "negative", "LABEL_1": "neutral", "LABEL_2": "positive"}

def classify_sentiment(texts, sentiment_pipe):
    results = sentiment_pipe(texts)
    return [label_map[r['label']] for r in results]

# === Streamlit App === #
st.set_page_config(page_title="BERTopic + Sentiment Explorer", layout="wide")
st.title("✈️ Airline Review Analyzer with BERTopic & Sentiment")

topic_model, sentiment_pipe = load_resources()
df = load_data()

# === Fit model once and cache === #
if "topics_fitted" not in st.session_state:
    with st.spinner("Fitting BERTopic model on full data..."):
        topics, probs = topic_model.fit_transform(df['customer_review'].tolist())
        df['topic'] = topics
        df['sentiment'] = classify_sentiment(df['customer_review'].tolist(), sentiment_pipe)
        st.session_state['topics_fitted'] = True
        st.session_state['df'] = df
        st.session_state['topic_model'] = topic_model
else:
    df = st.session_state['df']
    topic_model = st.session_state['topic_model']

# === BERTopic Summary Button === #
if st.button("Show Topic Insights"):
    topic_info = topic_model.get_topic_info()
    topic_summary = df.groupby('topic').agg(
        review_count=('customer_review', 'count'),
        positive=('sentiment', lambda x: (x == 'positive').mean()),
        neutral=('sentiment', lambda x: (x == 'neutral').mean()),
        negative=('sentiment', lambda x: (x == 'negative').mean()),
        top_sentiment=('sentiment', lambda x: x.value_counts().idxmax())
    ).reset_index().merge(topic_info[['Topic', 'Name']], left_on='topic', right_on='Topic').drop(columns=['Topic'])

    st.success("Topic modeling and sentiment classification loaded!")

    st.subheader("🔍 Top 10 Most Negative Topics")
    topic_summary_sorted = topic_summary.sort_values('negative', ascending=False).head(10)
    st.dataframe(topic_summary_sorted[['Name', 'negative', 'review_count']])

    plt.figure(figsize=(10,5))
    sns.barplot(data=topic_summary_sorted, x='Name', y='negative', palette='Reds_r')
    plt.xticks(rotation=45, ha='right')
    plt.title("Top 10 Topics by % Negative Sentiment")
    st.pyplot(plt.gcf())

    worst_topic = topic_summary_sorted.iloc[0]['Name']
    topic_id = topic_summary_sorted.iloc[0]['topic']
    st.markdown(f"### 👎 Negative Review Samples for Topic: **{worst_topic}**")
    sample_reviews = df[(df['topic'] == topic_id) & (df['sentiment'] == 'negative')]['customer_review'].sample(3)
    for r in sample_reviews:
        st.write(f"- {r[:300]}...")

# === Try Your Own Review === #
st.subheader("🧪 Try Your Own Review")
user_input = st.text_area("Enter a customer review", height=150)

if st.button("Analyze My Review"):
    topic_id, _ = topic_model.transform([user_input])
    topic_words = topic_model.get_topic(topic_id[0])
    topic_name = ", ".join([w for w, _ in topic_words]) if topic_words else "Unknown/Outlier"

    sentiment = sentiment_pipe([user_input])[0]
    st.markdown("**📝 Review:**")
    st.write(user_input)
    st.markdown(f"**🏷️ Topic:** {topic_name}")
    st.markdown(f"**Sentiment:** {label_map[sentiment['label']]} ({round(sentiment['score'], 2)})")


Overwriting app.py


In [4]:
!pip install streamlit transformers torch
!npm install -g localtunnel

⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧
added 22 packages in 3s
⠧
⠧3 packages are looking for funding
⠧  run `npm fund` for details
⠧

In [5]:
# Your public ip is the password to the localtunnel
!curl ipv4.icanhazip.com

34.125.129.36


In [ ]:
!streamlit run app.py & npx localtunnel --port 8501



⠙⠹⠸⠼
  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://34.125.129.36:8501

⠴⠦⠧⠇your url is: https://ten-ads-clean.loca.lt
2025-07-13 22:04:14.177769: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1752444254.202581    2452 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1752444254.210220    2452 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-07-13 22:04:14.236766: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following